In [34]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import torchinfo

sys.path.append(os.path.join(os.pardir, os.pardir))
from amlutils.task2.loading import load_train_set

from IPython.display import display

np.random.seed(42)

In [31]:
X_train_orig, y_train_orig = load_train_set(os.path.join(os.pardir, 'data'))
display(X_train_orig)

Epoch 0:   0%|          | 0/80 [11:15<?, ?it/s]


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x17832,x17833,x17834,x17835,x17836,x17837,x17838,x17839,x17840,x17841
id,,,,,,,,,,,,,,,,,,,,,
0,-64,-66,-69,-72,-75,-77,-80,-86,-89,-83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,505,500,496,492,487,480,475,476,483,495,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-21,-16,-12,-7,-3,0,1,2,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-211,-457,-635,-710,-715,-663,-573,-481,-401,-337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36,32,29,25,22,19,17,15,12,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,-285,-303,-334,-376,-413,-432,-443,-451,-460,-468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5113,50,51,50,48,46,44,42,39,36,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5114,-207,-225,-242,-258,-266,-271,-275,-279,-281,-284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_orig.fillna(0.0),
    y_train_orig,
    test_size=0.2,
    shuffle=False,
    stratify=None
)

def convert_to_tensor(X):
    X_tensor = torch.tensor(X.values, dtype=torch.float)
    # Unsqueeze X tensor to have another dimension representing the channel, this
    # is needed for convolutions.
    X_tensor = torch.unsqueeze(X_tensor, 1)
    return X_tensor

def build_data_loader(X, y):
    X_tensor = convert_to_tensor(X)
    y_tensor = torch.tensor(y.values)

    train_tensor = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset=train_tensor, batch_size=64, shuffle=True)

train_loader = build_data_loader(X_train, y_train)
valid_loader = build_data_loader(X_valid, y_valid)

In [69]:
num_samples = y_train.shape[0]
num_samples_per_class = y_train.value_counts().values

class_ratios = num_samples_per_class / num_samples
class_inverse_ratios = 1 / class_ratios

normalized_class_inverse_ratios = class_inverse_ratios / class_inverse_ratios.sum()

display(num_samples)
display(num_samples_per_class)
display(class_ratios)
display(class_inverse_ratios)
display(normalized_class_inverse_ratios)

4093

array([2436, 1168,  360,  129])

array([0.59516247, 0.28536526, 0.08795505, 0.03151722])

array([ 1.68021346,  3.50428082, 11.36944444, 31.72868217])

array([0.03479955, 0.07257851, 0.23547695, 0.65714498])

In [76]:
# Architecture based on: https://pythonwife.com/convolutional-autoencoders-opencv/

class ECGCNN(pl.LightningModule):

    def __init__(self) -> None:
        super().__init__()

        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=3, kernel_size=12, stride=2, dilation=3),
            nn.BatchNorm1d(num_features=3),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(in_channels=3, out_channels=6, kernel_size=6, stride=2, dilation=2),
            nn.BatchNorm1d(num_features=6),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(in_channels=6, out_channels=3, kernel_size=3, stride=2, dilation=1),
            nn.BatchNorm1d(num_features=3),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Flatten(),
            nn.LazyLinear(out_features=4)
        )

        # Dummy forward pass to initialize Lazy* layers.
        self.layers(torch.ones(10, 1, 17842))

    def forward(self, X):
        return F.softmax(self.layers(X), dim=1)

    def training_step(self, batch, batch_idx):
        X, y = batch
        # For cross-entropy loss, require that y.shape == (batch_size), but
        # y has shape (batch_size, 1) so squeeze out unnecessary dimension.
        y = torch.squeeze(y)

        y_pred = self.layers(X)
        loss = F.cross_entropy(
            y_pred,
            y,
            weight=torch.FloatTensor(normalized_class_inverse_ratios)
        )

        # Logging to TensorBoard by default
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        # For cross-entropy loss, require that y.shape == (batch_size), but
        # y has shape (batch_size, 1) so squeeze out unnecessary dimension.
        y = torch.squeeze(y)

        y_pred = self.layers(X)
        loss = F.cross_entropy(y_pred, y)

        self.log('valid_loss', loss)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        return optimizer


torchinfo.summary(ECGCNN(), input_size=(64, 1, 17842))

/usr/local/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Layer (type:depth-idx)                   Output Shape              Param #
ECGCNN                                   --                        --
├─Sequential: 1-1                        [64, 4]                   --
│    └─Conv1d: 2-1                       [64, 3, 8905]             39
│    └─BatchNorm1d: 2-2                  [64, 3, 8905]             6
│    └─ReLU: 2-3                         [64, 3, 8905]             --
│    └─Dropout: 2-4                      [64, 3, 8905]             --
│    └─MaxPool1d: 2-5                    [64, 3, 4452]             --
│    └─Conv1d: 2-6                       [64, 6, 2221]             114
│    └─BatchNorm1d: 2-7                  [64, 6, 2221]             12
│    └─ReLU: 2-8                         [64, 6, 2221]             --
│    └─Dropout: 2-9                      [64, 6, 2221]             --
│    └─MaxPool1d: 2-10                   [64, 6, 1110]             --
│    └─Conv1d: 2-11                      [64, 3, 554]              57
│    └─BatchNor

In [77]:
ecg_cnn = ECGCNN()

trainer = pl.Trainer(callbacks=[EarlyStopping(monitor='valid_loss')])
trainer.fit(ecg_cnn, train_loader, valid_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 3.6 K 
--------------------------------------
3.6 K     Trainable params
0         Non-trainable params
3.6 K     Total params
0.014     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s] 

/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 4: 100%|██████████| 80/80 [00:14<00:00,  5.47it/s, loss=0.809, v_num=14]


In [78]:
trainer.save_checkpoint('ecg-cnn-weighted-crossentropy.ckpt')

In [79]:
valid_score = f1_score(
    y_valid,
    ecg_cnn(convert_to_tensor(X_valid)).detach().numpy().argmax(axis=1),
    average='micro'
)
print(f'ECG-CNN validation F1 micro score: {valid_score}')

ECG-CNN validation F1 micro score: 0.314453125
